<br>
<br>
<br>

# Calculate Window Centroid 
This notebook is dedicated to computing the centroid of rectangular windows on 3D buildings.
<br>
<br>
<br>

In [ ]:
#Import modules:
import arcpy
import os

<br>
<br>

## Help functions

In [ ]:
def get_field_names(fc):
    
    #Get a list of fieldnames for input shapefile:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Return list with fieldnames:
    return f_names

In [ ]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the content of the corresponding Table Of Contents (TOC):
def get_TOC(fc):
    
    #Get a list of field-names:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Open shapefile and print every row in TOC:
    with arcpy.da.SearchCursor(fc, f_names) as cursor:
        
        for row in cursor:
            
            print(row)

In [ ]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column name whose values we want to extract and a boolean 
#(unique=True returns only unique values in column) and returns
#the content of the corresponding column in a list:
def get_TOC_column(fc, colname, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colname) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include unique values:
            if (unique):
                
                #Check that values is not already stored in the list:
                if row[0] not in ls:
                    
                    #Append value:
                    ls.append(row[0])
                    
            else:
                
                #Append column value to list:
                ls.append(row[0])
    
    #return list:
    return ls

In [ ]:
#Function that creates a new numeric field/column for a given 
#feature class 
#(feature_class-string, colname-string, datatype-string ("LONG"), numofvalues-integer, numofdecvalues-integer)
def create_long_field(fc, colname, datatype, numofvalues, numofdecvalues):
    
    #Extract field-names from shapefile:
    fieldnames = [f.name for f in arcpy.ListFields(fc)]
    
    #Check if "NN_dist" field already exists:
    if(colname not in fieldnames):

        #Add new field (distance to nearest neighbor) to shapefile: 
        try:
            arcpy.AddField_management(fc, colname, datatype, numofvalues, numofdecvalues)
            #print(colname+ "-field added to " + fc[0])

        except Exception as e:
            print("Error: " + e.args[0])
    
    
    

In [ ]:
#Function that reads and returns the name of the spatial reference
#system for the input feature layer:
def get_CRS(fc):
    
    #Extract spatial reference system information from feature class:
    spatial_ref = arcpy.Describe(fc).spatialReference
    
    #Return:
    return spatial_ref.name
    

<br>
<br>

## Compute window centroids - (main function)

In [ ]:
#Function that takes a multipatch layer of rectangular windows as input 
#and returns a new feature class with their centroids (point layer):
def compute_w_centroid(fc, vcs, out_file):
    
    #Convert window multipatch to points:
    w_points = arcpy.FeatureVerticesToPoints_management(fc)
    
    #Convert window points to lines:
    w_lines = arcpy.management.PointsToLine(Input_Features=w_points, Line_Field="ORIG_FID",
                                            Sort_Field="", Close_Line="NO_CLOSE")
    
    #Extract 3D geometry (Z: elevation for every window line):
    arcpy.CheckExtension("3D") 
    arcpy.CheckOutExtension("3D")
    arcpy.ddd.AddZInformation(w_lines, "Z_MEAN", 'NO_FILTER')
    arcpy.CheckInExtension("3D")
    
    #Add centroid fields to window-lines shapefile:
    create_long_field(w_lines, "C_X_coord", "LONG", 9, 2)
    create_long_field(w_lines, "C_Y_coord", "LONG", 9, 2)
    
    
    #Call function to calculate centroid:
    arcpy.management.CalculateGeometryAttributes(in_features=w_lines,
                                                 geometry_property=[["C_X_coord", "CENTROID_X"],
                                                                    ["C_Y_coord", "CENTROID_Y"]],
                                                 length_unit="", area_unit="",
                                                 coordinate_system=arcpy.SpatialReference(arcpy.Describe(fc).spatialReference.factoryCode, vcs),
                                                 coordinate_format="SAME_AS_INPUT")
    
    #Create separate shapefile (point layer) with window centroids:
    arcpy.management.XYTableToPoint(w_lines, out_file,
                                    x_field = "C_X_coord", 
                                    y_field = "C_Y_coord",
                                    z_field = "Z_MEAN",
                                    coordinate_system = arcpy.SpatialReference(arcpy.Describe(fc).spatialReference.factoryCode, vcs))
    
    
    #Rename field corresponding to centroid c-coordinate:
    create_long_field(out_file, "C_Z_coord", "LONG", 9, 2)
    arcpy.CalculateField_management(out_file, "C_Z_coord", "!Z_MEAN!", "PYTHON_9.3", "")
    arcpy.DeleteField_management(out_file, "Z_MEAN")
    arcpy.DeleteField_management(out_file, "Id")
    
    #Join input feature class with output feature class to maintain columns:
    arcpy.JoinField_management(out_file, 'ORIG_FID', fc, 'FID')
    
    #Delete help-files:
    #Check files in working folder:
    for file in os.listdir(working_environment):

        #Find temp help-files:
        if (file.endswith(".shp") & ("FeatureVerticesToPoin" in file) & (fc.split('.')[0] in file)):

            #Delete files:
            arcpy.Delete_management(file)


<br>
<br>

## Execute calculations

<br>
<br>

In [ ]:
#Set input variables:
building_number = 16
working_environment = r"C:\Users\Karolina.Pantazatou\Documents\ArcGIS\Projects\solkvarteret_hyllie_2\reproj"
input_feature_class = "windows.shp"
VCS = 5613 #vertical coordinate system (5613 corresponds to RH2000) 

# Set environment settings:
arcpy.env.workspace = working_environment

#Set output variables:
out_path = "C:\\Users\\Karolina.Pantazatou\\Documents\\ArcGIS\\Projects\\solkvarteret_hyllie_2\\window_centroids\\reproj\\"
out_file = "droppen_w_centroids.shp"
output = out_path+out_file

In [ ]:
# Get description of input feature class:
desc = arcpy.da.Describe(input_feature_class)

In [ ]:
#Call function to compute window centroids:
compute_w_centroid(input_feature_class, VCS, output)

<br>
<br>

## Check output

In [ ]:
get_field_names(output)

In [ ]:
get_TOC(output)

In [ ]:
get_field_names(input_feature_class)

In [ ]:
get_TOC(input_feature_class)

<br>
<br>

## Reference links:

- Extensions:
    - https://gis.stackexchange.com/questions/54044/arcgis-error-000824-the-tool-is-not-licensed-in-arcpy
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/checkextension.htm
    
    
- Feature vertices to Points (Data Management):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/feature-vertices-to-points.htm


- Add Z Information (3D Analyst):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/3d-analyst/add-z-information.htm


- Centroid calculation (alternative for 2D datasets):
    - https://stackoverflow.com/questions/23020659/fastest-way-to-calculate-the-centroid-of-a-set-of-coordinate-tuples-in-python-wi
    
    
- Spatial Reference:
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm
    

- Delete Shapefile:
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm
    
    
- Find files in directory:
    - https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python
    
    
- Join Field (Data Management):
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/join-field.htm

<br>
<br>